# Azure Threat Hunting

** by Maarten Goet, MVP & RD **

Install the Kqlmagic extension

(optional, because Azure Notebooks container has this installed by default)

In [ ]:
!pip install Kqlmagic --no-cache-dir --upgrade

Load the Kqlmagic extension

In [ ]:
%reload_ext Kqlmagic

Connect to our Log Analytics backend (in this case: Azure Security Center)

In [6]:
%kql loganalytics://tenant='...';clientsecret='...';workspace='...';alias='...'

<IPython.core.display.Javascript object>

Query ASC for any network connections this VM has

In [ ]:
%kql VMConnection | where Computer == ‘APPSERVER’

Filter the query so that we only see connections to IP's outside of The Netherlands

In [ ]:
%%kql 
    VMConnection
    | where Computer == 'APPSERVER'
    | where Direction == 'outbound'
    | where RemoteCountry != 'Netherlands'

Put the results into a dataframe

In [ ]:
connections = _.to_dataframe()

Load the Pandas extensions, then import a list of TOR nodes (IP addresses) from this URL into a list called 'torlist'

In [11]:
import pandas as pd
torlist = pd.read_csv('https://www.dan.me.uk/torlist',header=0,names=["DestinationIp"])

Compare the KQL output to the TOR list and see if we have any match

In [ ]:
connections.merge(torlist, on=‘DestinationIp’)

Query ASC for locations of outbound IP's and put them into a list called 'locations'

In [12]:
%%kql locations <<
    VMConnection
    | where Computer == "APPSERVER"
    | where Direction == "outbound"
    | where RemoteCountry != "Netherlands"
    | distinct RemoteLongitude, RemoteLatitude // only get unique records
    | project RemoteLongitude, RemoteLatitude  // only show these two colums

Install the Basemap extension

In [ ]:
!conda install basemap -y

Load Basemap to do some geographical visualizations

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

map = Basemap()
map.drawcoastlines()

plt.show()